# CLEAR iLO LOGS

<div class="alert alert-block alert-info">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>for clearing out the ilo event log and management log at server side!!</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "#" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    
</div>

<div class = "alert alert-block alert-success">
<h1>1</h1>
In the beginning, we are having some import statements that are actually providing us with the required modules and libraries.
for proper execution of our code
</div>

In [3]:
import sys
import json
import getpass
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

<div class="alert alert-block alert-success">
    <h1>2</h1>
This cell gives overview about of the function used to clear event logs such as IML and IEL logs.       
The IML provides a record of historical events that have occurred on the server and the IEL is the iLO event log provides record of significant events detected by iLO.
    
1. When the resource directory is disabled then we simply navigate through the API for obtaining the relevant uri.
    The series of HTTP get requests aims to extracts <b>log_services_response</b> so as to get the path of the directory for which IEL and IML logs need to be cleared.
    <br>
2. Else if the resource directory enabled then rather we can simply extract the uri by <b>#LogService.</b>attribute inside the <b>instance['odatatype']</b> dictionary and again make a HTTP  get request to obtain the required uri.
    <br>
3. After obtaining the uri we just need to make changes on the same location using a post request so as to clear IEL and IML logs successfully. For this body dictionary contains the attribute to be changed and IEL/IML are searched in the path of <b>clear_log_services_uri </b> and the respective changes are made by <b>redfishobj.post(path,body)</b>.
    <br>
    
4. Later we also check for the response returned by the server side. If the response is anything outside of status code with 200 then error message gets rendered on the webpage else the success message is printed out with the response dictionary returned by the server.

</div>

In [4]:
def clear_ilo_event_log(_redfishobj, clear_IML_IEL):

    clear_log_services_uri = []

    resource_instances = redfishobj.get_resource_directory()
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
        log_services_uri = systems_members_response.obj['LogServices']['@odata.id']
        log_services_response = _redfishobj.get(log_services_uri)
        log_services_uris = log_services_response.obj['Members']
        for log_services_uri in log_services_uris:
            log_services_response = _redfishobj.get(log_services_uri['@odata.id'])
            clear_log_services_uri.append(log_services_response.obj['Actions']\
                                                                ['#LogService.ClearLog']['target'])
    else:
        for instance in resource_instances:
            #Use Resource directory to find the relevant URI
            if '#LogService.' in instance['@odata.type']:
                log_service_uri = instance['@odata.id']
                clear_log_services_uri.append(_redfishobj.get(log_service_uri).dict['Actions']\
                                                                ['#LogService.ClearLog']['target'])

    if clear_log_services_uri:
        body = {"Action": "LogService.ClearLog"}
        for path in clear_log_services_uri:
            if ("IEL" in clear_IML_IEL and "IEL" in path) or ("IML" in clear_IML_IEL and \
                                                                                    "IML" in path):
                if "IEL" in path:
                    sys.stdout.write("Clearing IEL log.\n")
                else:
                    sys.stdout.write("Clearing IML log.\n")
                resp = _redfishobj.post(path, body)
            else:
                continue
            #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended
            #info error message to see what went wrong
            if resp.status == 400:
                try:
                    print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                    sort_keys=True))
                except Exception as excp:
                    sys.stderr.write("A response error occurred, unable to access iLO "\
                                     "Extended Message Info...\n")
            elif resp.status != 200:
                sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
            else:
                print("Success!\n")
                print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class="alert alert-block alert-success">
<h1>3</h1>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE.<br>
    <ul>
        <li>Once you enter everything a redfish obejct is created with the help of RedfishClient class and a session is created and now you will be logged in to the server</li>
        <li>If server is down or Unreachable we get server not reachable message!</li>
        <li>Else if no error is encountered a function call is made i.e. clear_ilo_event_log(REDFISHOBJ, CLEAR_IEL_IML)
        that is responsible for handling clearing all the IEL and IML logs at the server side. </li>.
        <li>Lastly, Redfishobj.logout logs us out of the system and ends the session.</li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD = getpass.getpass("Enter password: ")

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False
    CLEAR_IML_IEL = "IML" # provide either the string IML or IEL

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    clear_ilo_event_log(REDFISHOBJ, CLEAR_IML_IEL)
    REDFISHOBJ.logout()

<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>